<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/061_Expert_Consultants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Enhancing Invoice Processing with Expert Tools

Introducing expert consultations to handle two critical tasks:

- **Categorizing Expenditures**: The agent will consult an expert to classify each invoice into one of 20 predefined spending categories based on a one-sentence description.
- **Checking Purchasing Rules**: The agent will validate whether the invoice follows company purchasing policies.

By using expert tools, we enable our agent to dynamically invoke specialized knowledge without hardcoding every rule, keeping the architecture clean and adaptable.

## Step 1: Creating the Invoice Categorization Expert

We’ll start by defining an expert who specializes in classifying expenditures. This expert will take a one-sentence description of the invoice and return the best-fitting category from our predefined list.


In [ ]:
@register_tool(tags=["invoice_processing", "categorization"])
def categorize_expenditure(action_context: ActionContext, description: str) -> str:
    """
    Categorize an invoice expenditure based on a short description.

    Args:
        description: A one-sentence summary of the expenditure.

    Returns:
        A category name from the predefined set of 20 categories.
    """
    categories = [
        "Office Supplies", "IT Equipment", "Software Licenses", "Consulting Services",
        "Travel Expenses", "Marketing", "Training & Development", "Facilities Maintenance",
        "Utilities", "Legal Services", "Insurance", "Medical Services", "Payroll",
        "Research & Development", "Manufacturing Supplies", "Construction", "Logistics",
        "Customer Support", "Security Services", "Miscellaneous"
    ]

    return prompt_expert(
        action_context=action_context,
        description_of_expert="A senior financial analyst with deep expertise in corporate spending categorization.",
        prompt=f"Given the following description: '{description}', classify the expense into one of these categories:\n{categories}"
    )

## Step 2: Creating the Purchasing Rules Expert

To ensure that our purchasing rules expert is always using the latest policy guidelines, we will implement it to read the purchasing rules from a file on disk before validating an invoice. This approach makes our system dynamic—we don’t need to hardcode the rules into the agent, and they can be updated easily without modifying the code. Also, we don’t need a special format for the rules, the human-readable policy becomes the logic!

First, let’s assume that the purchasing rules are stored in a simple text file located at `"config/purchasing_rules.txt"`. The expert will read these rules, insert them into the prompt, and then validate the invoice accordingly.


In [ ]:
@register_tool(tags=["invoice_processing", "validation"])
def check_purchasing_rules(action_context: ActionContext, invoice_data: dict) -> dict:
    """
    Validate an invoice against company purchasing policies.

    Args:
        invoice_data: Extracted invoice details, including vendor, amount, and line items.

    Returns:
        A dictionary indicating whether the invoice is compliant, with explanations.
    """
    # Load the latest purchasing rules from disk
    rules_path = "config/purchasing_rules.txt"

    try:
        with open(rules_path, "r") as f:
            purchasing_rules = f.read()
    except FileNotFoundError:
        purchasing_rules = "No rules available. Assume all invoices are compliant."

    return prompt_expert(
        action_context=action_context,
        description_of_expert="A corporate procurement compliance officer with extensive knowledge of purchasing policies.",
        prompt=f"""
        Given this invoice data: {invoice_data}, check whether it complies with company purchasing rules.
        The latest purchasing rules are as follows:

        {purchasing_rules}

        Identify any violations or missing requirements. Respond with:
        - "compliant": true or false
        - "issues": A brief explanation of any problems found
        """
    )

## Example: Updating Purchasing Rules Without Code Changes

Let’s assume our purchasing rules file contains:

- All purchases over \$5,000 require pre-approval.
- IT equipment purchases must be from approved vendors.
- Travel expenses must include a justification.
- Consulting fees over $10,000 require an SOW (Statement of Work).



In [ ]:
{
    "invoice_number": "7890",
    "date": "2025-03-10",
    "vendor": "Tech Solutions Inc.",
    "total_amount": 6000,
    "line_items": [
        {"description": "High-end workstation", "quantity": 1, "total": 6000}
    ]
}
Expert Output

{
    "compliant": false,
    "issues": "This purchase exceeds $5,000 but lacks required pre-approval."
}


This is an example of how expert personas can be operationalized in a real system — and a great blend of **structured data**, **dynamic reasoning**, and **human-readable business logic**.

---

### 🔑 1. **Two Types of Expert Personas for Two Distinct Functions**

You're seeing **two personas** doing different jobs:

* **`categorize_expenditure`**: Uses a financial analyst persona to select one of 20 predefined spending categories from a simple description.
* **`check_purchasing_rules`**: Uses a compliance officer persona to read *real policy text* from a file and reason over structured invoice data.

> 💡 **Focus Insight**: You're not hardcoding logic — you're injecting expertise into prompts and allowing the LLM to *apply* the rules instead of *encoding* them manually in code.

---

### 🧱 2. **Composable Tool Pattern**

Both tools follow the same architectural design:

* Defined expert description
* Focused prompt
* Cleanly wrapped in a reusable tool

> 🛠️ This makes them *plug-and-play*. You can drop these tools into any agent system without modifying the agent itself.

---

### 📂 3. **External Rules = Dynamic Behavior**

In `check_purchasing_rules`, rules are stored in a **text file**:

```python
with open(rules_path, "r") as f:
    purchasing_rules = f.read()
```

* This makes policy logic editable by non-programmers.
* The agent automatically updates its behavior when the file changes.

> 🧠 This is **"prompt-driven software design"**: logic lives in text, not in code.

---

### 🎭 4. **Persona Prompt Design**

Both tools use expert descriptions that act as **role selectors** for the LLM:

* `"A senior financial analyst..."` for categorization
* `"A corporate procurement compliance officer..."` for validation

> 🧠 This triggers the right *style of reasoning* without needing a massive prompt.

---

### ✅ 5. **Structured Input + Structured Output**

Look at how clearly structured the data flow is:

* Input: a `description` or an `invoice_data` dictionary
* Output: a single `category` or a compliance report dict (`{compliant: true/false, issues: ...}`)

> 🧩 This keeps the entire system **interoperable** with APIs, databases, spreadsheets, etc.

---

### 🧪 6. **Real-World Validation Example**

This test case shows how rules are enforced dynamically:

```json
{
  "compliant": false,
  "issues": "This purchase exceeds $5,000 but lacks required pre-approval."
}
```

> 🔍 The LLM didn't just repeat the rules — it **applied them** based on context and returned structured feedback.

---

### ✅ What You Should Focus On:

| Area                          | Why It Matters                                                       |
| ----------------------------- | -------------------------------------------------------------------- |
| **Persona Design**            | Use concise but rich expert profiles to activate the right reasoning |
| **Prompt Injection of Rules** | Keep logic outside of code when possible — editable, updatable       |
| **Structured Data Flow**      | Ensures outputs can plug into downstream systems                     |
| **Dynamic Flexibility**       | These tools adapt without changing agent code                        |
| **Tool Modularity**           | Design each function as a single-purpose module                      |





### 💡 Why Use Specialized Personas?

Each persona:

* **Encapsulates a way of thinking** tailored to a domain (finance vs. compliance).
* Keeps prompts **focused and readable**, avoiding one huge overloaded agent.
* Is easy to **swap, improve, or upgrade** independently.

This makes your system more like a **collaborative team of experts** — not a single overloaded generalist.

---

### 📝 Why Put Rules in an External File?

* ✅ **Editable by non-devs** (managers, auditors, analysts)
* 🔁 **Live updates** — change a line in the file, and the agent instantly reflects it
* 🔍 **Transparency** — anyone can read and audit the actual rules
* 🔧 **No redeploy required** — update rules, not code

> It’s a kind of *softcoding* where the logic lives in human-readable form, not Python or Java.

---

### 🧠 Why It Works So Well with LLMs

LLMs *thrive* on:

* Natural language inputs (like your purchasing policies)
* Contextual reasoning (like applying “rules” to structured invoice data)
* Roleplay (personas bring consistent behavior)

So you're designing in a way that **amplifies what LLMs are best at** — rather than forcing them into rigid, traditional code logic.

---

If you keep combining:

* Modular **tools**
* Real-world **personas**
* Externalized **business logic**

…you're not just using LLMs — you're building **intelligent, evolving systems** that mirror how real-world teams operate.






### 🧩 What You’re Really Building Is a Cognitive Architecture

You’ve already started designing a system that…

* **Separates concerns** (extraction vs. validation vs. categorization)
* **Encapsulates reasoning** in modular, swappable parts
* **Bridges human-readable policy** with machine-executable logic
* **Lets LLMs operate as adaptable collaborators**, not just black boxes

This is *not* traditional software engineering. It’s something **newer, closer to organizational design** — with tools that think like experts, guided by simple inputs and evolving policy.

---

### 🧠 Zooming Out: What This Teaches You

As you master this style of agent design, you're learning to:

* Think in **systems of logic + language**
* Model real-world **professional roles** as interactive tools
* Design pipelines that **reason, adapt, and evolve**

And this mindset transfers **far beyond invoices** — to legal workflows, research assistants, onboarding agents, AI tutors, scientific assistants, and more.

So yes — this might be "just an invoice system," but you’re already practicing **high-level agent architecture design.**

